In [3]:
# Load the necessary package
library(readr)

# Specify the file path
file_path <- "../data/us_ind_by_month.csv"

# Read the CSV file
data <- read_csv(file_path)

# Load the necessary packages
library("ggplot2")
library("scales")
library("ggpubr")
library("ggthemr")
library("ggpmisc")
# install ggthemr using github
devtools::install_github("cttobin/ggthemr")
ggthemr('flat')
library("egg")
library("extrafont")
library("ggtext")
library("lubridate")
library("zoo")
library("ggtext")
library("stringr")
library("ggtext")
library("scales")
library("directlabels")
library("lubridate")

New names:
• `` -> `...1`
Rows: 6048 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Month, NAICS 2022 3-Digit Industry Group (Name)
dbl (7): ...1, Year, Year-Month, NAICS 2022 3-Digit Industry Group, Percent,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in library("ggpubr"): there is no package called ‘ggpubr’


In [2]:
install.packages('ggpubr')

Installing package into ‘/home/davidcai/R/x86_64-redhat-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘lme4’, ‘corrplot’, ‘car’, ‘ggrepel’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’


Warning message in install.packages("ggpubr"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘lme4’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘pbkrtest’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘car’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘rstatix’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘ggpubr’ had non-zero exit status”


In [ ]:
# we group the data by year and 
# first multiply N by Percent to arrive at the number of WFH jobs
# second we add up the number of WFH jobs for each year
# third we devide the number of WFH jobs for each year by the total
# number of jobs for that year to get the percentage of WFH jobs

In [ ]:
library(dplyr)

# Assuming your data frame is named 'data' and it has columns 'Year', 'N', and 'Percent' # nolint
data_by_year_NAICS <- data %>% # nolint
  group_by(`Year`,`NAICS 2022 3-Digit Industry Group`,`NAICS 2022 3-Digit Industry Group (Name)`) %>% # nolint
  mutate(WFH_jobs = N * Percent / 100) %>%
  summarise(Total_WFH_jobs = sum(WFH_jobs),
            Total_jobs = sum(N),
            Percent_WFH_jobs = Total_WFH_jobs / Total_jobs * 100)

`summarise()` has grouped output by 'Year', 'NAICS 2022 3-Digit Industry
Group'. You can override using the `.groups` argument.


In [ ]:
# keep only data from 2019
data_by_year_NAICS_2019 <- data_by_year_NAICS %>% filter(Year == 2019)
# sort by Percent_WFH_jobs
data_by_year_NAICS_2019 <- data_by_year_NAICS_2019 %>% arrange(desc(Percent_WFH_jobs))
# create a rank column
data_by_year_NAICS_2019$rank <- 1:nrow(data_by_year_NAICS_2019)
# keep only the naics code and rank columns
data_by_year_NAICS_2019 <- data_by_year_NAICS_2019 %>% ungroup() %>% select(`NAICS 2022 3-Digit Industry Group`,rank)
# merge in into the grouped dataset
data_by_year_NAICS <- merge(data_by_year_NAICS,data_by_year_NAICS_2019,by = "NAICS 2022 3-Digit Industry Group")

In [ ]:
names(data_by_year_NAICS)

[1] "NAICS 2022 3-Digit Industry Group"       
[2] "Year"                                    
[3] "NAICS 2022 3-Digit Industry Group (Name)"
[4] "Total_WFH_jobs"                          
[5] "Total_jobs"                              
[6] "Percent_WFH_jobs"                        
[7] "rank"

In [ ]:
# keep only year 2019/2023/2024
data_by_year_NAICS <- data_by_year_NAICS %>% filter(Year %in% c(2019,2023,2024))

In [ ]:
p <- ggplot(data_by_year_NAICS, aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4","2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(face = "bold", size = 24, hjust = 0, family = "sans", colour = "black"),  # Center and bold the title with size 24
    legend.position = "none",  # Remove legend
    axis.text = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis text size and font
    axis.title = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis title size and font
    axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, family = "sans", colour = "black"),  # Rotate x-axis text for better readability
    plot.caption = ggtext::element_textbox(hjust = 0, halign = 0, family = "sans", size = 20, colour = "black", lineheight = 1.2, width = unit(0.95, "npc")),  # Use element_textbox for caption with size 20, wrapped and left-aligned text
    plot.caption.position = "plot",
    axis.title.x = element_blank(), # remove the x-axis title
    panel.background = element_rect(fill = "white"),  # Set panel background to white
    aspect.ratio = 3/5,  # Set aspect ratio
    axis.line = element_line(color = "black"),  # Set axis lines to black
    axis.line.x = element_line(color = "black", size = 1),  # Thicken x-axis line
    axis.line.y = element_line(color = "black", size = 1),  # Thicken y-axis line
    axis.line.x.top = element_blank(),  # Remove top x-axis line
    axis.line.y.right = element_blank(),  # Remove right y-axis line
    axis.ticks = element_line(color = "black", size = 1),  # Ensure axis ticks are visible
    axis.ticks.length = unit(0.3, "cm")  # Lengthen the ticks for better visibility
  )
ggsave("../output_blogpost_1/figure2.pdf", plot = p, width = 16, height = 32, dpi = 300)

ERROR: Error in loadNamespace(x): there is no package called ‘ggtext’


In [ ]:
# now make the two subgraphs
# first include on the the top 5 industries
p <- ggplot(data_by_year_NAICS %>% filter(rank <= 5), aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Top 5 - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4", "2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(face = "bold", size = 24, hjust = 0, family = "sans", colour = "black"),  # Center and bold the title with size 24
    legend.position = "none",  # Remove legend
    axis.text = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis text size and font
    axis.title = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis title size and font
    axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, family = "sans", colour = "black"),  # Rotate x-axis text for better readability
    plot.caption = ggtext::element_textbox(hjust = 0, halign = 0, family = "sans", size = 20, colour = "black", lineheight = 1.2, width = unit(0.95, "npc")),  # Use element_textbox for caption with size 20, wrapped and left-aligned text
    plot.caption.position = "plot",
    axis.title.x = element_blank(), # remove the x-axis title
    panel.background = element_rect(fill = "white"),  # Set panel background to white
    aspect.ratio = 3/5,  # Set aspect ratio
    axis.line = element_line(color = "black"),  # Set axis lines to black
    axis.line.x = element_line(color = "black", size = 1),  # Thicken x-axis line
    axis.line.y = element_line(color = "black", size = 1),  # Thicken y-axis line
    axis.line.x.top = element_blank(),  # Remove top x-axis line
    axis.line.y.right = element_blank(),  # Remove right y-axis line
    axis.ticks = element_line(color = "black", size = 1),  # Ensure axis ticks are visible
    axis.ticks.length = unit(0.3, "cm")  # Lengthen the ticks for better visibility
  )
ggsave("../output_blogpost_1/figure2_top5.pdf", plot = p, width = 16, height = 10, dpi = 300)

# now make the two subgraphs
# first include on the the bottom 5 industries

p <- ggplot(data_by_year_NAICS %>% filter(rank >= 92), aes(x = Percent_WFH_jobs,
 y = reorder(`NAICS 2022 3-Digit Industry Group (Name)`, -1*rank),
 fill = factor(Year))) +
  geom_bar(stat = "identity", position = position_dodge(width = 0.8)) +
  labs(
    title = "Percent of job ads offering remote/hybrid vary widely by Industry - Bottom 5 - Sorted by 2019 Shares",
    x = "Percent",
    y = NULL,
    fill = "Year'"
  ) +
  theme_minimal() +
  scale_fill_manual(values = c("2024" = "#F8766D", "2023" = "#00BFC4", "2019" = "#7CAE00")) +
  theme(
    plot.title = element_text(face = "bold", size = 24, hjust = 0, family = "sans", colour = "black"),  # Center and bold the title with size 24
    legend.position = "none",  # Remove legend
    axis.text = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis text size and font
    axis.title = element_text(face = "bold", size = 20, family = "sans", colour = "black"),  # Adjust axis title size and font
    axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1, family = "sans", colour = "black"),  # Rotate x-axis text for better readability
    plot.caption = ggtext::element_textbox(hjust = 0, halign = 0, family = "sans", size = 20, colour = "black", lineheight = 1.2, width = unit(0.95, "npc")),  # Use element_textbox for caption with size 20, wrapped and left-aligned text
    plot.caption.position = "plot",
    axis.title.x = element_blank(), # remove the x-axis title
    panel.background = element_rect(fill = "white"),  # Set panel background to white
    aspect.ratio = 3/5,  # Set aspect ratio
    axis.line = element_line(color = "black"),  # Set axis lines to black
    axis.line.x = element_line(color = "black", size = 1),  # Thicken x-axis line
    axis.line.y = element_line(color = "black", size = 1),  # Thicken y-axis line
    axis.line.x.top = element_blank(),  # Remove top x-axis line
    axis.line.y.right = element_blank(),  # Remove right y-axis line
    axis.ticks = element_line(color = "black", size = 1),  # Ensure axis ticks are visible
    axis.ticks.length = unit(0.3, "cm")  # Lengthen the ticks for better visibility
  )
ggsave("../output_blogpost_1/figure2_bottom5.pdf", plot = p, width = 16, height = 10, dpi = 300)
